In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
%cd ..

/Users/javier.burroni/deeppl/deepppl/deepppl


In [4]:
import pyro
import dppl

In [5]:
import torch

In [6]:
sns.set_style('whitegrid')


### Basic example showing the interface of *DeepPPL*
Similar to the coin model but this time using a `guide` to do stochastic variational inference


In [7]:
model_src = """
data {
  int<lower=0,upper=1> x[10];
}
parameters {
  real<lower=0,upper=1> theta;
}


guide {
  real<lower=0>  alpha_q;
  real<lower=0>  beta_q;
  alpha_q = tensor(15.0);
  beta_q = tensor(15.0);
  theta ~ beta(alpha_q, beta_q);
}

model {
  theta ~ beta(tensor(5.0), tensor(5.0));
  for (i in 1:10)
    x[i] ~ bernoulli(theta);
}

"""

In [8]:
model = dppl.DppplModel(model_code = model_src)

In [9]:
model._guide.__globals__['tensor'] = lambda x: torch.tensor(x, dtype=torch.float)

In [10]:
svi = model.svi()

In [11]:
x = torch.Tensor([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0])

In [12]:
for step in range(10000):
    svi.step(x)
    if step % 100 == 0:
        print('.', end='')

....................................................................................................

In [13]:
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

In [14]:
alpha_q, beta_q

(11.449324607849121, 17.959613800048828)

In [15]:
alpha_q/(alpha_q + beta_q)

0.3893144475005713

To do more experiments, the store must be cleared

In [112]:
pyro.clear_param_store()